# Winery classification with the multivariate Gaussian

In this notebook, we return to winery classification, using the full set of 13 features.

## 1. Load in the data 

As usual, we start by loading in the Wine data set. Make sure the file `wine.data.txt` is in the same directory as this notebook.

Recall that there are 178 data points, each with 13 features and a label (1,2,3). As before, we will divide this into a training set of 130 points and a test set of 48 points.

In [1]:
# Standard includes
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
# Useful module for dealing with the Gaussian density
from scipy.stats import norm, multivariate_normal 

In [2]:
# Load data set.
data = np.loadtxt('wine.data.txt', delimiter=',')
# Names of features
featurenames = ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash','Magnesium', 'Total phenols', 
                'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 
                'OD280/OD315 of diluted wines', 'Proline']
# Split 178 instances into training set (trainx, trainy) of size 130 and test set (testx, testy) of size 48
np.random.seed(0)
perm = np.random.permutation(178)
trainx = data[perm[0:130],1:14]
trainy = data[perm[0:130],0]
testx = data[perm[130:178], 1:14]
testy = data[perm[130:178],0]

## 2. Fit a Gaussian generative model

We now define a function that fits a Gaussian generative model to the data.
For each class (`j=1,2,3`), we have:
* `pi[j]`: the class weight
* `mu[j,:]`: the mean, a 13-dimensional vector
* `sigma[j,:,:]`: the 13x13 covariance matrix

This means that `pi` is a 4x1 array (Python arrays are indexed starting at zero, and we aren't using `j=0`), `mu` is a 4x13 array and `sigma` is a 4x13x13 array.

- 每一类拟合一个高斯模型

In [3]:
def fit_generative_model(x,y):
    k = 3  # labels 1,2,...,k
    d = (x.shape)[1]  # number of features
    mu = np.zeros((k+1,d))
    sigma = np.zeros((k+1,d,d))
    pi = np.zeros(k+1)
    for label in range(1,k+1):
        indices = (y == label)  # y仅仅用在计算类的比例，以及区分不同类型的样本上，计算均值和协方差矩阵没有贡献
        mu[label] = np.mean(x[indices,:], axis=0)
        sigma[label] = np.cov(x[indices,:], rowvar=0, bias=1)
        pi[label] = float(sum(indices))/float(len(y))
    return mu, sigma, pi

In [4]:
# Fit a Gaussian generative model to the training data
mu, sigma, pi = fit_generative_model(trainx,trainy)

In [12]:
mu

array([[  0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00,
          0.00000000e+00],
       [  1.37853488e+01,   2.02232558e+00,   2.42790698e+00,
          1.68813953e+01,   1.05837209e+02,   2.85162791e+00,
          2.99627907e+00,   2.89069767e-01,   1.93069767e+00,
          5.63023256e+00,   1.06232558e+00,   3.16674419e+00,
          1.14190698e+03],
       [  1.23109259e+01,   1.91925926e+00,   2.22703704e+00,
          1.96759259e+01,   9.58333333e+01,   2.30129630e+00,
          2.10907407e+00,   3.47407407e-01,   1.65722222e+00,
          3.18370370e+00,   1.04129630e+00,   2.76666667e+00,
          5.25981481e+02],
       [  1.31596970e+01,   3.37727273e+00,   2.40090909e+00,
          2.08939394e+01,   9.90303030e+01,   1.67606061e+00,
          7.57272727e-01,   4.59696970e-01,   1.208

In [26]:
xx, xy = np.meshgrid([1,2,3], [1,2,3])
xx.ravel(), xy.ravel()

(array([1, 2, 3, 1, 2, 3, 1, 2, 3]), array([1, 1, 1, 2, 2, 2, 3, 3, 3]))

In [28]:
sigma[1,xx.ravel(), xy.ravel()].reshape(3,3)

array([[ 0.43132948, -0.00977188,  0.23815955],
       [-0.00977188,  0.03677469,  0.23526339],
       [ 0.23815955,  0.23526339,  6.04011898]])

In [52]:
sigma[1,[1,2,3],[1,2,3]]

array([ 0.43132948,  0.03677469,  6.04011898])

## 3. Use the model to make predictions on the test set

<font color="magenta">**For you to do**</font>: Define a general purpose testing routine that takes as input:
* the arrays `pi`, `mu`, `sigma` defining the generative model, as above
* the test set (points `tx` and labels `ty`)
* a list of features `features` (chosen from 0-12)

It should return the number of mistakes made by the generative model on the test data, *when restricted to the specified features*. For instance, using the just three features 2 (`'Ash'`), 4 (`'Magnesium'`) and 6 (`'Flavanoids'`) results in 7 mistakes (out of 48 test points), so 

        `test_model(mu, sigma, pi, [2,4,6], testx, testy)` 

should print 7/48.

**Hint:** The way you restrict attention to a subset of features is by choosing the corresponding coordinates of the full 13-dimensional mean and the appropriate submatrix of the full 13x13 covariance matrix.

- 训练好模型后，均值和协方差矩阵就固定下来了，如果只需要部分特征，只需要从参数中选择相应的参数就可以了
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.norm.html

In [65]:
# Now test the performance of a predictor based on a subset of features
def test_model(mu, sigma, pi, features, tx, ty):
    ###
    ### Your code goes here
    ###
    k = 3  # number of labels
    n_test = len(ty)
    score = np.zeros((n_test, k+1))
    n_f = len(features)
    xx, xy = np.meshgrid(features, features)
    # xx.ravel(), xy.ravel()
    for i in range(n_test):
        for label in range(1,k+1):
            #将乘积转化成log下的加法，在拟合多变量高斯模型时，需要使用multivariate_normal，且需要传入相关feature的协方差矩阵
            score[i,label] = np.log(pi[label]) + multivariate_normal.logpdf(tx[i,features], 
                                                                            mean=mu[label,features], 
                                                                            cov=sigma[label,xx,xy].reshape(n_f, n_f))
            # norm.logpdf(tx[i,features], mu[label,features], np.sqrt(sigma[label,features,features]))
    # predict
    predictions = np.argmax(score[:,1:k+1], axis=1) + 1
    errors = np.sum(predictions!=ty)
    print('test error using feature ({}) {}'.format(','.join([str(i) for i in features]), 
                                                    [featurenames[i] for i in features]) + ': {} / 48'.format(errors))

### <font color="magenta">Fast exercises</font>

*Note down the answers to these questions. You will need to enter them as part of this week's assignment.*

Exercise 1. How many errors are made on the test set when using the single feature 'Ash'?

In [66]:
','.join([str(i) for i in [1,2,3]])

'1,2,3'

In [67]:
test_model(mu, sigma, pi, [2], testx, testy)

test error using feature (2) ['Ash']: 29 / 48


Exercise 2. How many errors when using 'Alcohol' and 'Ash'?

In [68]:
test_model(mu, sigma, pi, [0,2], testx, testy)

test error using feature (0,2) ['Alcohol', 'Ash']: 12 / 48


Exercise 3. How many errors when using 'Alcohol', 'Ash', and 'Flavanoids'?

In [69]:
test_model(mu, sigma, pi, [0,2,6], testx, testy)

test error using feature (0,2,6) ['Alcohol', 'Ash', 'Flavanoids']: 3 / 48


Exercise 4. How many errors when using all 13 features?

In [70]:
test_model(mu, sigma, pi, range(0,13), testx, testy)

test error using feature (0,1,2,3,4,5,6,7,8,9,10,11,12) ['Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']: 2 / 48


Exercise 5. In lecture, we got somewhat different answers to these questions. Why do you think that might be?